In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import*
from tensorflow.keras.layers import*
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import*
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.image import ResizeMethod
import albumentations as A

import glob
import tifffile as tiff
import cv2
from skimage.io import imsave, imread, imshow
from skimage import color
from PIL import Image
from skimage.transform import resize
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.metrics import MeanIoU, Accuracy
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
color_dict = {
    0: [60,16,152], # Building
    1: [132,41,246], # Land
    2: [110,193,228], # road
    3: [254,221,58], # Vegetation
    4: [226,169,41], # water
    5: [155,155,155], # Unlabeled
    # 6: [0,0,0] # Padding
}

def rgb_to_onehot(rgb_arr, color_dict):
    num_classes = len(color_dict)
    shape = rgb_arr.shape[:2] + (num_classes,)
    arr = np.zeros(shape,dtype=np.int8)

    for i, cls in enumerate(color_dict):
        arr[:,:,i] = np.all(rgb_arr.reshape((-1,3))==color_dict[i],axis=1).reshape(shape[:2])
        arr = np.array(arr)
    return arr

def onehot_to_rgb(onehot, color_dict):
    output = np.zeros(onehot.shape[:2] + (3,))
    mask = np.argmax(onehot,axis=2)
    for k in color_dict.keys():
        output[mask==k] = color_dict[k]
    return output.astype(int)

def get_dataset(X_dir, y_dir, color_dict, flag):
    num_samples = len(X_dir)
    X = []
    y = []

    for i in range(num_samples):
        image = plt.imread(X_dir[i])
        mask = plt.imread(y_dir[i])

        X.append(np.array(tf.image.resize_with_crop_or_pad(image, 512, 512))) 
        y.append(rgb_to_onehot((np.array(tf.image.resize_with_crop_or_pad(mask[:,:,0:3], 512, 512)))*255, color_dict)) 
        # if size is large -> centrally crop
        # if size is less -> centrally pad with zeros

        if flag == 0:
            for j in range(8): # creating 8 augmented copies for each train image
                transform = augment(512, 512)
                transformed = transform(image=image, mask=mask)
                transformed_image = transformed['image']
                transformed_mask = transformed['mask']

                X.append(transformed_image)
                y.append(rgb_to_onehot((transformed_mask[:,:,0:3])*255, color_dict))

    return np.array(X), np.array(y)

def augment(width, height):
    transform = A.Compose([
      A.RandomCrop(width=width, height=height, p=1.0),
      A.HorizontalFlip(p=1.0),
      A.VerticalFlip(p=1.0),
      A.Rotate(limit=[60, 300], p=1.0, interpolation=cv2.INTER_NEAREST),
      A.RandomBrightnessContrast(brightness_limit=[-0.2, 0.3], contrast_limit=0.2, p=1.0),
      A.OneOf([
          A.CLAHE (clip_limit=1.5, tile_grid_size=(8, 8), p=0.5),
          A.GridDistortion(p=0.5),
          A.OpticalDistortion(distort_limit=1, shift_limit=0.5, interpolation=cv2.INTER_NEAREST, p=0.5),
      ], p=1.0),
    ], p=1.0)

    return transform

In [ ]:
x_train = glob.glob('/kaggle/input/DubaiSat-Semantic-Segmentation/UAE Semantic segmentation dataset/Train/images/*.jpg')
y_train = glob.glob('/kaggle/input/DubaiSat-Semantic-Segmentation/UAE Semantic segmentation dataset/Train/masks/*.png')

x_test = glob.glob('/kaggle/input/DubaiSat-Semantic-Segmentation/UAE Semantic segmentation dataset/Test/images/*.jpg')
y_test = glob.glob('/kaggle/input/DubaiSat-Semantic-Segmentation/UAE Semantic segmentation dataset/Test/masks/*.png')

x_train.sort()
y_train.sort()

x_test.sort()
y_test.sort()

print(len(x_train),len(y_train),len(x_test),len(y_test))

x_train, y_train = get_dataset(x_train, y_train, color_dict, 0)
x_test, y_test = get_dataset(x_test, y_test, color_dict, 1)

print(x_train.shape ,y_train.shape,x_test.shape,y_test.shape)

In [ ]:
for i in range(0,2):
    fig, ax = plt.subplots(nrows=2, ncols=9)
    fig.tight_layout()
    fig.set_size_inches(20, 20)

    for j in range(0,9):
        ax[0][j].imshow(x_train[9*i + j])
        ax[0][j].axis('off')
        ax[0][j].set_title('train_img_'+str(i))
        ax[1][j].imshow(onehot_to_rgb(y_train[9*i + j], color_dict),cmap='gray')
        ax[1][j].axis('off')
        ax[1][j].set_title('train_gt')

    fig.subplots_adjust(wspace=0.2,hspace=-0.86)
    plt.show()

In [ ]:
img_width = 512
img_height = 512
img_channels = 3

inputs = Input((img_height, img_width, img_channels))

c1 = Conv2D(16, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = Conv2D(16, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2,2)) (c1)

c2 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = MaxPooling2D((2,2)) (c2)

c3 = Conv2D(64, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = Conv2D(64, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = MaxPooling2D((2,2)) (c3)

c4 = Conv2D(128, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = Dropout(0.2)(c4)
c4 = Conv2D(128, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = MaxPooling2D((2,2)) (c4)

c5 = Conv2D(256, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = Dropout(0.4)(c5)
c5 = Conv2D(256, (3,3), activation = 'relu', kernel_initializer='he_normal', padding='same')(c5)

# Decoder

u6 = Conv2DTranspose(128,(2,2), strides=(2,2), padding = 'same') (c5)
u6 = concatenate([u6,c4])
c6 = Conv2D(128,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(u6)
c6 = Dropout(0.2)(c6)
c6 = Conv2D(128,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(c6)

u7 = Conv2DTranspose(64,(2,2), strides=(2,2), padding = 'same') (c6)
u7 = concatenate([u7,c3])
c7 = Conv2D(64,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(u7)
c7 = Conv2D(64,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(c7)

u8 = Conv2DTranspose(32,(2,2), strides=(2,2), padding = 'same') (c7)
u8 = concatenate([u8,c2])
c8 = Conv2D(32,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(u8)
c8 = Conv2D(32,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(c8)

u9 = Conv2DTranspose(16,(2,2), strides=(2,2), padding = 'same') (c8)
u9 = concatenate([u9,c1],axis = 3)
c9 = Conv2D(16,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(u9)
c9 = Conv2D(16,(3,3), activation = 'relu', kernel_initializer= 'he_normal', padding ='same')(c9)

outputs = Conv2D(6,(1,1), activation='softmax')(c9)

model = Model(inputs=[inputs], outputs = [outputs])

opt = Adam(learning_rate = 0.0001)
model.compile(optimizer=opt, loss= 'categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

In [ ]:
model_checkpoint = ModelCheckpoint('/kaggle/working/UAE_Unet.h5', monitor='val_loss', save_best_only=True)
history = model.fit(x_train, y_train, validation_data=(x_test,y_test),epochs=100, batch_size=4, verbose=1, callbacks=[model_checkpoint])

In [ ]:
f, ax = plt.subplots()
ax.plot(history.history['loss'], )
ax.plot(history.history['val_loss'], '--')
ax.legend(['Train Loss', 'Validation Loss'])
ax.set_title('Training/Validation Loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.show()

In [ ]:
f, ax = plt.subplots()
ax.plot(history.history['accuracy'], )
ax.plot(history.history['val_accuracy'], '--')
ax.legend(['Train accuracy', 'Validation accuracy'])
ax.set_title('Training/Validation accuracy per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('accuracy')
plt.show()

In [ ]:
model.load_weights('/kaggle/working/UAE_Unet.h5')
y_pred = model.predict(x_test)

In [ ]:
for i in range(1):
    fig, ax = plt.subplots(3, 9)
    fig.tight_layout()
    fig.set_size_inches(20, 20)

    for j in range(0,9):
        ax[0][j].imshow(x_test[9*i + j])
        ax[0][j].axis('off')
        ax[0][j].set_title('test_img_'+str(i))
        ax[1][j].imshow(onehot_to_rgb(y_test[9*i + j], color_dict),cmap='gray')
        ax[1][j].axis('off')
        ax[1][j].set_title('test_gt')
        ax[2][j].imshow(onehot_to_rgb(y_pred[9*i + j], color_dict),cmap='gray')
        ax[2][j].axis('off')
        ax[2][j].set_title('test_pred')

    fig.subplots_adjust(wspace=0.2,hspace=-0.86)
    plt.show()

In [ ]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_inception_resnetv2_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained InceptionResNetV2 Model """
    encoder = InceptionResNetV2(include_top=False, weights="imagenet", input_tensor=inputs)

    """ Encoder """
    s1 = encoder.get_layer("input_1").output           ## (512 x 512)

    s2 = encoder.get_layer("activation").output        ## (255 x 255)
    s2 = ZeroPadding2D(( (1, 0), (1, 0) ))(s2)         ## (256 x 256)

    s3 = encoder.get_layer("activation_3").output      ## (126 x 126)
    s3 = ZeroPadding2D((1, 1))(s3)                     ## (128 x 128)

    s4 = encoder.get_layer("activation_74").output      ## (61 x 61)
    s4 = ZeroPadding2D(( (2, 1),(2, 1) ))(s4)           ## (64 x 64)

    """ Bridge """
    b1 = encoder.get_layer("activation_161").output     ## (30 x 30)
    b1 = ZeroPadding2D((1, 1))(b1)                      ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)
    
    """ Output """
    dropout = Dropout(0.3)(d4)
    outputs = Conv2D(6, 1, padding="same", activation="softmax")(dropout)

    model = Model(inputs, outputs, name="InceptionResNetV2-UNet")
    return model

model = build_inception_resnetv2_unet(input_shape = (512, 512, 3))
model.compile(optimizer=Adam(lr = 0.0001), loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

In [ ]:
model_checkpoint = ModelCheckpoint('/kaggle/working/UAE_inception.h5', monitor='val_loss', save_best_only=True)
history = model.fit(x=x_train, y=y_train, validation_data=(x_test,y_test),epochs=100, batch_size=4, verbose=1, callbacks=[model_checkpoint])

In [ ]:
f, ax = plt.subplots()
ax.plot(history.history['loss'], )
ax.plot(history.history['val_loss'], '--')
ax.legend(['Train Loss', 'Validation Loss'])
ax.set_title('Training/Validation Loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.show()

In [ ]:
f, ax = plt.subplots()
ax.plot(history.history['accuracy'], )
ax.plot(history.history['val_accuracy'], '--')
ax.legend(['Train accuracy', 'Validation accuracy'])
ax.set_title('Training/Validation accuracy per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('accuracy')
plt.show()

In [ ]:
model.load_weights('/kaggle/working/UAE_inception.h5')
y_pred = model.predict(x_test)

In [ ]:
for i in range(1):
    fig, ax = plt.subplots(nrows=3, ncols=9)
    fig.tight_layout()
    fig.set_size_inches(20, 20)

    for j in range(0,9):
        ax[0][j].imshow(x_test[9*i + j])
        ax[0][j].axis('off')
        ax[0][j].set_title('test_img_'+str(i))
        ax[1][j].imshow(onehot_to_rgb(y_test[9*i + j], color_dict),cmap='gray')
        ax[1][j].axis('off')
        ax[1][j].set_title('test_gt')
        ax[2][j].imshow(onehot_to_rgb(y_pred[9*i + j], color_dict),cmap='gray')
        ax[2][j].axis('off')
        ax[2][j].set_title('test_pred')

    fig.subplots_adjust(wspace=0.2,hspace=-0.86)
    plt.show()